# Nested logit

In [1]:
import numpy as np
import os
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from IPython.display import display, HTML

In [4]:
thepath = os.path.join(os.getcwd(),'..')
travelmode =  pd.read_csv(os.path.join(thepath,'data_mec_optim/demand_travelmode/travelmodedata.csv'))

In [5]:
travelmode.head()

,individual,mode,choice,wait,vcost,travel,gcost,income,size
0,1,air,no,69,59,100,70,35,1
1,1,train,no,34,31,372,71,35,1
2,1,bus,no,35,25,417,70,35,1
3,1,car,yes,0,10,180,30,35,1
4,2,air,no,64,58,68,68,30,2


In [10]:
# Change choice from string to int
lb = LabelEncoder() 
travelmode['choice'] = lb.fit_transform(travelmode['choice'])

In [11]:
travelmode.head()

,individual,mode,choice,wait,vcost,travel,gcost,income,size
0,1,air,0,69,59,100,70,35,1
1,1,train,0,34,31,372,71,35,1
2,1,bus,0,35,25,417,70,35,1
3,1,car,1,0,10,180,30,35,1
4,2,air,0,64,58,68,68,30,2


In [12]:
nobs = travelmode.shape[0]
ncols = travelmode.shape[1]
nbchoices = 4
ninds = int(nobs/nbchoices)

In [13]:
muhat_i_y = travelmode['choice'].values.reshape(ninds,4).T
muhat_iy = muhat_i_y.flatten()

In [14]:
s_y = travelmode.groupby(['mode']).mean()['choice'].to_frame().sort_index()
s_y

,choice
mode,
air,0.276190
bus,0.142857
car,0.280952
train,0.300000


In [15]:
Ulogit = np.log(s_y/s_y.loc[['car']].values[0])
Ulogit

,choice
mode,
air,-0.017094
bus,-0.676340
car,0.000000
train,0.065597


In [16]:
l = [0.5, 0.5]
nest = Ulogit.index.isin(['car'])
Unocar = s_y[~nest].apply(np.log).multiply(l[0]).add((1-l[0])*np.sum(s_y[~nest]).apply(np.log))
display(Unocar)
Ucar = s_y[nest].apply(np.log).multiply(l[1]).add((1-l[1])*np.sum(s_y[nest]).apply(np.log))
display(Ucar)

,choice
mode,
air,-0.808246
bus,-1.137869
train,-0.766900


,choice
mode,
car,-1.26957


Systematic utilities (nested logit):

In [17]:
Unested = pd.concat([Unocar,Ucar]).add(-Ucar.values[0]).sort_index()
Unested

,choice
mode,
air,0.461324
bus,0.131701
car,0.000000
train,0.502670


Choice probabilities within nocar nest (predicted vs observed):

In [18]:
display(Unested[~nest].multiply(1/l[0]).apply(np.exp).divide(np.sum(Unested[~nest].multiply(1/l[0]).apply(np.exp))))
display(s_y[~nest]/np.sum(s_y[~nest]))

,choice
mode,
air,0.384106
bus,0.198675
train,0.417219


,choice
mode,
air,0.384106
bus,0.198675
train,0.417219


Choice probabilities of car nest (predicted vs observed):

In [19]:
display(1/(np.sum(Unested[~nest].multiply(1/l[0]).apply(np.exp))**(l[0])+1))
display(s_y.loc[['car']].values[0])

choice    0.280952
dtype: float64

array([0.28095238])